## Tensor factorization with AMP with p = 3 

In [1]:
import torch
from math import sqrt


# Priors
def prior_gb(A, B, prmts):
    """Compute f_a and f_c for Gauss-Bernoulli prior"""

    rho, mu, sig = prmts

    m = (B * sig + mu) / (1. + A * sig)
    v = sig / (1 + A * sig)
    keep = (rho + (1 - rho) * torch.sqrt(1. + A * sig) *
            torch.exp(-.5 * m ** 2 / v + .5 * mu ** 2 / sig))
    p_s = rho / keep

    a = p_s * m
    c = p_s * v + p_s * (1. - p_s) * m ** 2
    return a, torch.mean(c)


# Solver
def amp(T, Delta3,
        prior=prior_gb, prior_prmts=None, true_coef=None,
        max_iter=250, tol=1e-13, verbose=1):
    """Iterate VAMP equations"""

    N, _, _ = T.shape

    # Initialize variables
    B3 = torch.zeros(N, 1)
    A3 = 0.
    a = torch.randn(N, 1)
    a_old = torch.zeros(N, 1)
    c = 1.
    S2 = sqrt(2)
    SN = sqrt(N)
    K = S2 / (Delta3 * N)
    if verbose > 0:
        print("time ; convergence mse, Matrix-mmse ")
    for t in range(max_iter):
        # Messages/estimates on x from likelihood
        B3 = (K * torch.transpose(a, 0, 1)@T@a).view(N, -1)
        B3 = B3 - (2 / Delta3) * c * (torch.mean(a * a)) * a_old
        A3 = (1. / Delta3) * torch.mean(a * a)**2
        a_old = torch.clone(a)
        
        a, cc = prior(A3, B3, prior_prmts)        
        c = torch.mean(cc)
        
        a = 0.5 * a + 0.5 * a_old

        # Compute metrics
        conv = torch.mean((a - a_old) ** 2)
        overl = torch.abs(torch.mean (a * X0))
        mse = torch.mean((a - true_coef) ** 2) if true_coef is not None else 0.
        Mmse = torch.mean((a@torch.transpose(a, 0, 1) - (true_coef@torch.transpose(true_coef, 0, 1)))**2) if true_coef is not None else 0.
        if verbose > 0:
            print("t = %d; conv = %g, mse = %g, Mmse = %g" % (t, conv, mse, Mmse))
        if conv < tol:
            break
    return a

In [34]:
import torch
# We keep CPU by default, if you want GPU uncomment the next line
# torch.set_default_tensor_type('torch.cuda.FloatTensor')#GPU by default

from math import sqrt
import time

N = 300
DELTA3 = 10

X0 = torch.randn(N,1)
Y = X0@torch.transpose(X0, 0, 1)
T = Y.view(N, N, 1)@torch.transpose(X0, 0, 1)

R0 = torch.randn(N,N,N)
R1 = torch.transpose(R0,0,1)
R2 = torch.transpose(R0,0,2)
R3 = torch.transpose(R0,1,2)
R4 = torch.transpose(R1,1,2)
R5 = torch.transpose(R2,1,2)
RUMORE = (R0 + R1 + R2 + R3 + R4 +R5)/sqrt(6)

T = (sqrt(2) / N) * T + RUMORE * sqrt(DELTA3)
t = time.time()
Xhat = amp(T, DELTA3,
       prior=prior_gb, prior_prmts=(1, 0, 1),
           true_coef=X0, max_iter=250)
elapsed = time.time() - t

time ; convergence mse, Matrix-mmse 
t = 0; conv = 0.326125, mse = 1.25492, Mmse = 1.17077
t = 1; conv = 0.0382279, mse = 1.10711, Mmse = 1.11161
t = 2; conv = 0.0154844, mse = 1.05422, Mmse = 1.10114
t = 3; conv = 0.00757589, mse = 1.03887, Mmse = 1.09965
t = 4; conv = 0.00331316, mse = 1.03822, Mmse = 1.09961
t = 5; conv = 0.00122141, mse = 1.04143, Mmse = 1.09967
t = 6; conv = 0.000388137, mse = 1.04442, Mmse = 1.0997
t = 7; conv = 0.000111197, mse = 1.04637, Mmse = 1.09971
t = 8; conv = 2.99101e-05, mse = 1.04747, Mmse = 1.09971
t = 9; conv = 7.76742e-06, mse = 1.04806, Mmse = 1.09971
t = 10; conv = 1.9799e-06, mse = 1.04836, Mmse = 1.09971
t = 11; conv = 4.9985e-07, mse = 1.04852, Mmse = 1.09971
t = 12; conv = 1.2558e-07, mse = 1.04859, Mmse = 1.09971
t = 13; conv = 3.14725e-08, mse = 1.04863, Mmse = 1.09971
t = 14; conv = 7.87786e-09, mse = 1.04865, Mmse = 1.09971
t = 15; conv = 1.97068e-09, mse = 1.04866, Mmse = 1.09971
t = 16; conv = 4.92824e-10, mse = 1.04867, Mmse = 1.09971
t